In [ ]:
# Task 2: CSP con Backtracking, Beam y Local Search
# Definición de variables y restricciones para el problema de programación de exámenes

# Definición de los exámenes y días
exams = ['Examen1', 'Examen2', 'Examen3', 'Examen4', 'Examen5', 'Examen6', 'Examen7']
days = ['lunes', 'martes', 'miércoles']
students = ['Estudiante1', 'Estudiante2', 'Estudiante3', 'Estudiante4']

# Restricciones del problema
# Cada estudiante tiene los exámenes que toma, y debemos evitar que se asignen en el mismo día
# Exámenes que los estudiantes toman
student_courses = {
    'Estudiante1': ['Examen1', 'Examen2', 'Examen3'],
    'Estudiante2': ['Examen2', 'Examen4', 'Examen5'],
    'Estudiante3': ['Examen3', 'Examen6', 'Examen7'],
    'Estudiante4': ['Examen4', 'Examen5', 'Examen6']
}

# Función para verificar si una asignación es válida
def is_valid(assignment, constraints):
    # Verifica que los exámenes sean asignados a días diferentes
    days_assigned = list(assignment.values())
    if len(days_assigned) != len(set(days_assigned)):  # Si hay días duplicados, no es válido
        return False
    
    # Verifica que ningún estudiante tenga más de un examen en el mismo día
    for student, courses in student_courses.items():
        student_days = [assignment[course] for course in courses if course in assignment]
        if len(student_days) != len(set(student_days)):  # Si el estudiante tiene más de un examen en el mismo día
            return False
        
    return True

# Función para preprocesar el problema con Arc Consistency (AC-3)
def ac3(exams, days, constraints):
    queue = []
    for exam in exams:
        for student, courses in constraints.items():
            if exam in courses:
                queue.append((exam, student))  # Añadir todos los arcos a la cola para su revisión
    
    while queue:
        exam, student = queue.pop(0)
        for day in days:
            # Comprobar que la asignación de días no violen las restricciones de los estudiantes
            # Si se elimina una opción para el estudiante, agregamos los vecinos a la cola
            pass  # Implementar el algoritmo AC-3 de acuerdo a la reducción de dominios
    
    return exams, days  # Debería retornar las variables actualizadas

# Algoritmo de Backtracking Search Optimizado con Heurística
def backtracking(assignment, exams, days, constraints):
    # Heurística: Ordenar los exámenes por el número de restricciones en el dominio
    if len(assignment) == len(exams):
        return assignment  # Si todos los exámenes están asignados, se retorna la solución

    # Elige el examen con el dominio más pequeño (MRV - Minimum Remaining Values)
    exam = select_exam(assignment, exams, days, constraints)

    for day in days:
        assignment[exam] = day
        if is_valid(assignment, constraints):  # Verificar si la asignación es válida
            result = backtracking(assignment, exams, days, constraints)  # Llamada recursiva
            if result is not None:
                return result
        del assignment[exam]  # Retrocedemos
    return None

# Heurística para seleccionar el examen con el dominio más pequeño
def select_exam(assignment, exams, days, constraints):
    unassigned_exams = [exam for exam in exams if exam not in assignment]
    # Ordenar los exámenes por el tamaño del dominio (MRV)
    return min(unassigned_exams, key=lambda exam: len(days))  # Elige el examen con menos días disponibles

# Algoritmo de Beam Search
def beam_search(exams, days, beam_width, constraints):
    beams = [({}, 0)]  # (asignaciones, puntuación)
    
    while beams:
        new_beams = []
        for assignment, score in beams:
            for exam in exams:  # Iterar sobre los exámenes
                for day in days:
                    new_assignment = assignment.copy()
                    new_assignment[exam] = day
                    if is_valid(new_assignment, constraints):
                        new_beams.append((new_assignment, score + heuristic(new_assignment)))
        
        # Mantener los mejores "beam_width" caminos
        beams = sorted(new_beams, key=lambda x: x[1])[:beam_width] 
        
    return beams[0][0]  # Devuelve la mejor asignación encontrada

# Heurística para Beam Search
def heuristic(assignment):
    # Heurística simple basada en la cantidad de restricciones violadas
    return sum([1 for day in assignment.values() if day in assignment.values()])

# Algoritmo de Local Search
def local_search(exams, days, constraints):
    current_solution = generate_initial_solution(exams, days)
    while True:
        neighbors = generate_neighbors(current_solution, exams, days)
        best_neighbor = min(neighbors, key=lambda x: heuristic(x))
        if heuristic(best_neighbor) >= heuristic(current_solution):
            break  # Si no se mejora, terminamos
        current_solution = best_neighbor
    return current_solution

# Función para generar la solución inicial
def generate_initial_solution(exams, days):
    return {exam: days[i % len(days)] for i, exam in enumerate(exams)}

# Función para generar vecinos en Local Search
def generate_neighbors(solution, exams, days):
    neighbors = []
    for exam in exams:
        for day in days:
            new_solution = solution.copy()
            new_solution[exam] = day
            neighbors.append(new_solution)
    return neighbors

# Evaluar el tiempo de ejecución de cada algoritmo
import time

# Medir el tiempo para Backtracking
start_time = time.time()
backtracking_solution = backtracking({}, exams, days, student_courses)
backtracking_time = time.time() - start_time
print(f"Backtracking time: {backtracking_time} seconds, Solution: {backtracking_solution}")

# Medir el tiempo para Beam Search
start_time = time.time()
beam_solution = beam_search(exams, days, beam_width=3, constraints=student_courses)
beam_time = time.time() - start_time
print(f"Beam Search time: {beam_time} seconds, Solution: {beam_solution}")


# Medir el tiempo para Local Search
start_time = time.time()
local_solution = local_search(exams, days, student_courses)
local_time = time.time() - start_time
print(f"Local Search time: {local_time} seconds, Solution: {local_solution}")

# Conclusiones 


KeyboardInterrupt: 